# Largest Lyapunov exponent calculation

### Algorithm

In [68]:
import numpy as np

In [69]:
# Logistic map
def logistic_map(n=2**15, r=4, x0=.4):
    x = np.zeros(n)
    x[0] = x0
    for i in range(n-1):
        x[i+1] = r*x[i]*(1-x[i])
    return x


# Henon map
def henon_map(n=1000000, a=1.4, b=0.3, x0=.4):
    x = np.zeros(n)
    x[0] = x0
    for i in range(1, len(x)):
        x[i] = 1 - a * x[i-1] ** 2 + b * x[i-1]
    return x


# Lorenz ts
def lorenz_ts(N=None):
    x = np.array([])
    i = 0
    with open("lorenz.txt") as f:
        for line in f:
            x = np.append(x, float(line))
            i += 1
            if N is not None and i == N:
                break
    return x

sine_data = np.sin(np.arange(0,1000,.01))

lorenz = lorenz_ts()
logistic = logistic_map()
henon = henon_map()

In [83]:
np.diag(np.ones(5) * np.inf)

array([[inf,  0.,  0.,  0.,  0.],
       [ 0., inf,  0.,  0.,  0.],
       [ 0.,  0., inf,  0.,  0.],
       [ 0.,  0.,  0., inf,  0.],
       [ 0.,  0.,  0.,  0., inf]])

In [111]:
test = np.array([1, 2, 3, 4])
test[test > 2] = 2
test

array([1, 2, 2, 2])

In [177]:
from sklearn.metrics.pairwise import euclidean_distances

def largest_exponent(series: np.array, J: int, m: int, t: float, trajectory_len: int = 20):
    # compute shape values
    N = series.shape[0]
    M = N - (m-1) * J

    # reconstruct with lag algorithm
    x = np.zeros((M, m))
    for i in range(M):
        indexes = np.ones(m) * i + np.arange(m) * J
        x[i] = series[indexes.astype(int)]
    
    # Find nearest neighbor
    distances = euclidean_distances(x)
    neighbors = np.argmin(distances + np.diag(np.ones(M) * np.inf), axis=1)

    # mean rate of separation
    y = np.zeros(trajectory_len)
    for i in range(trajectory_len):
        neighbors_i = neighbors[:M-i] + i
        neighbors_i[neighbors_i >= M] = (M - 1)
        separation = distances[(np.arange(M - i) + i, neighbors_i)]
        separation = separation[neighbors[:M-i] + i < M]
        separation = separation[separation != 0]
        
        if separation.shape[0] == 0:
            y[i] = np.inf
        else:
            y[i] = np.log(separation).mean() / t

    y = y[np.isfinite(y)]
    slope, _ = np.polyfit(np.arange(1, len(y) + 1), y, 1)
    return slope

In [181]:
print(f"{largest_exponent(logistic[:501], J=1, m=2, t=1, trajectory_len=48):.3f}")

0.066


In [169]:
def complexity_embedding(signal, delay=1, dimension=3, show=False, **kwargs):
    """**Time-delay Embedding of a Signal**

    Time-delay embedding is one of the key concept of complexity science. It is based on the idea
    that a dynamical system can be described by a vector of numbers, called its *'state'*, that
    aims to provide a complete description of the system at some point in time. The set of all
    possible states is called the *'state space'*.

    Takens's (1981) embedding theorem suggests that a sequence of measurements of a dynamic system
    includes in itself all the information required to completely reconstruct the state space.
    Time-delay embedding attempts to identify the state *s* of the system at some time *t* by
    searching the past history of observations for similar states, and, by studying the evolution
    of similar states, infer information about the future of the system.

    **Attractors**

    How to visualize the dynamics of a system? A sequence of state values over time is called a
    trajectory. Depending on the system, different trajectories can evolve to a common subset of
    state space called an attractor. The presence and behavior of attractors gives intuition about
    the underlying dynamical system. We can visualize the system and its attractors by plotting the
    trajectory of many different initial state values and numerically integrating them to
    approximate their continuous time evolution on discrete computers.

    Parameters
    ----------
    signal : Union[list, np.array, pd.Series]
        The signal (i.e., a time series) in the form of a vector of values. Can also be a string,
        such as ``"lorenz"`` (Lorenz attractor), ``"rossler"`` (Rössler attractor), or
        ``"clifford"`` (Clifford attractor) to obtain a pre-defined attractor.
    delay : int
        Time delay (often denoted *Tau* :math:`\\tau`, sometimes referred to as *lag*) in samples.
        See :func:`complexity_delay` to estimate the optimal value for this parameter.
    dimension : int
        Embedding Dimension (*m*, sometimes referred to as *d* or *order*). See
        :func:`complexity_dimension` to estimate the optimal value for this parameter.
    show : bool
        Plot the reconstructed attractor. See :func:`complexity_attractor` for details.
    **kwargs
        Other arguments to be passed to :func:`complexity_attractor`.

    Returns
    -------
    array
        Embedded time-series, of shape ``length - (dimension - 1) * delay``

    See Also
    ------------
    complexity_delay, complexity_dimension, complexity_attractor

    Examples
    ---------
    **Example 1**: Understanding the output

    .. ipython

      import neurokit2 as nk

      # Basic example
      signal = [1, 2, 3, 2.5, 2.0, 1.5]
      embedded = nk.complexity_embedding(signal, delay = 2, dimension = 2)
      embedded

    The first columns contains the beginning of the signal, and the second column contains the
    values at *t+2*.

    **Example 2**: 2D, 3D, and "4D" Attractors. Note that 3D attractors are slow to plot.

    .. ipython

      # Artifical example
      signal = nk.signal_simulate(duration=4, sampling_rate=200, frequency=5, noise=0.01)

      @savefig p_complexity_embedding1.png scale=100%
      embedded = nk.complexity_embedding(signal, delay=50, dimension=2, show=True)
      @suppress
      plt.close()

    .. ipython

      @savefig p_complexity_embedding2.png scale=100%
      embedded = nk.complexity_embedding(signal, delay=50, dimension=3, show=True)
      @suppress
      plt.close()

    .. ipython

      @savefig p_complexity_embedding3.png scale=100%
      embedded = nk.complexity_embedding(signal, delay=50, dimension=4, show=True)
      @suppress
      plt.close()

    In the last 3D-attractor, the 4th dimension is represented by the color.

    **Example 3**: Attractor of heart rate

      ecg = nk.ecg_simulate(duration=60*4, sampling_rate=200)
      peaks, _ = nk.ecg_peaks(ecg, sampling_rate=200)
      signal = nk.ecg_rate(peaks, sampling_rate=200, desired_length=len(ecg))

      @savefig p_complexity_embedding4.png scale=100%
      embedded = nk.complexity_embedding(signal, delay=250, dimension=2, show=True)
      @suppress
      plt.close()

    References
    -----------
    * Gautama, T., Mandic, D. P., & Van Hulle, M. M. (2003, April). A differential entropy based
      method for determining the optimal embedding parameters of a signal. In 2003 IEEE
      International Conference on Acoustics, Speech, and Signal Processing, 2003. Proceedings.
      (ICASSP'03). (Vol. 6, pp. VI-29). IEEE.
    * Takens, F. (1981). Detecting strange attractors in turbulence. In Dynamical systems and
      turbulence, Warwick 1980 (pp. 366-381). Springer, Berlin, Heidelberg.

    """

    N = len(signal)

    # Sanity checks
    if dimension * delay > N:
        raise ValueError(
            "NeuroKit error: complexity_embedding(): dimension * delay should be lower than",
            " the length of the signal.",
        )
    if delay < 1:
        raise ValueError("NeuroKit error: complexity_embedding(): 'delay' has to be at least 1.")

    Y = np.zeros((dimension, N - (dimension - 1) * delay))
    for i in range(dimension):
        Y[i] = signal[i * delay : i * delay + Y.shape[1]]
    embedded = Y.T

    return embedded



def _complexity_lyapunov_rosenstein(
    signal, delay=1, dimension=2, separation=1, len_trajectory=20, show=False, **kwargs
):
    # 1. Check that sufficient data points are available
    # Minimum length required to find single orbit vector
    min_len = (dimension - 1) * delay + 1
    # We need len_trajectory orbit vectors to follow a complete trajectory
    min_len += len_trajectory - 1
    # we need tolerance * 2 + 1 orbit vectors to find neighbors for each
    min_len += separation * 2 + 1

    # Embedding
    embedded = complexity_embedding(signal, delay=delay, dimension=dimension)
    m = len(embedded)

    # Construct matrix with pairwise distances between vectors in orbit
    dists = euclidean_distances(embedded)

    for i in range(m):
        # Exclude indices within separation
        dists[i, max(0, i - separation) : i + separation + 1] = np.inf

    # Find indices of nearest neighbours
    ntraj = m - len_trajectory + 1
    min_dist_indices = np.argmin(
        dists[:ntraj, :ntraj], axis=1
    )  # exclude last few indices
    min_dist_indices = min_dist_indices.astype(int)

    # Follow trajectories of neighbour pairs for len_trajectory data points
    trajectories = np.zeros(len_trajectory)
    for k in range(len_trajectory):
        divergence = dists[(np.arange(ntraj) + k, min_dist_indices + k)]
        dist_nonzero = np.where(divergence != 0)[0]
        if len(dist_nonzero) == 0:
            trajectories[k] = -np.inf
        else:
            # Get average distances of neighbour pairs along the trajectory
            trajectories[k] = np.mean(np.log(divergence[dist_nonzero]))

    divergence_rate = trajectories[np.isfinite(trajectories)]

    # LLE obtained by least-squares fit to average line
    slope, intercept = np.polyfit(
        np.arange(1, len(divergence_rate) + 1), divergence_rate, 1
    )

    # Store info
    parameters = {
        "Trajectory_Length": len_trajectory,
        "Divergence_Rate": divergence_rate,
    }

    if show is True:
        plt.plot(np.arange(1, len(divergence_rate) + 1), divergence_rate)
        plt.axline(
            (0, intercept), slope=slope, color="orange", label="Least-squares Fit"
        )
        plt.ylabel("Divergence Rate")
        plt.title(f"Largest Lyapunov Exponent (slope of the line) = {slope:.3f}")
        plt.legend()

    return slope, parameters

In [188]:
_complexity_lyapunov_rosenstein(lorenz[:5001], 11, 5, len_trajectory=10)

(0.10081046006089485,
 {'Trajectory_Length': 10,
  'Divergence_Rate': array([-3.20168663, -2.78025586, -2.63505089, -2.49701362, -2.28973965,
         -2.14620778, -2.13358299, -2.1820157 , -2.19914652, -2.19424653])})